# Neural Network in PyTorch

## Import Packages

In [37]:
import torch                                  # Uses Entire PyTorch Library
import torch.nn as nn                         # Imports PyTorch's Neural Network Toolkit
import torch.nn.functional as functions       # Has common NN functions like ReLu and Sigmoid
import torch.optim as optimizer               # Brings in Optimizers to update model params after backpropogation
from torch.utils.data import DataLoader       # Helps load our train/test data into our model

import numpy as np                            # Geenral math library we may need later on
from torchvision import datasets, transforms  # Brings in example datasets and tools to format them
from torchsummary import summary              # Tool to show the architecture of our network


## Define Dataset and Hyperparameters

In [38]:
# define training hyperparameters
n_epochs = 5
batch_size_train = 64
batch_size_test = 64
learning_rate = 0.001

# Load the MNIST dataset
train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transforms.ToTensor())
test_dataset = datasets.MNIST('./data', train=False, download=True, transform=transforms.ToTensor())

# Define your data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size_train, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size_test, shuffle=False)


## Define Neural Network Architecture

In [39]:
# Define the model architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = functions.relu(functions.max_pool2d(self.conv1(x), 2))
        x = functions.relu(functions.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = functions.relu(self.fc1(x))
        x = functions.dropout(x, training=self.training)
        x = self.fc2(x)
        x = functions.log_softmax(x, dim=1)
        return x

In [40]:
# create network
model = Net()

# uncomment to print network summary
summary(model, (1, 28, 28), device="cpu")

# define loss function and optimizer
optimizer = optimizer.Adam(model.parameters(), lr=learning_rate)
loss_function = nn.CrossEntropyLoss()

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 24, 24]             260
            Conv2d-2             [-1, 20, 8, 8]           5,020
         Dropout2d-3             [-1, 20, 8, 8]               0
            Linear-4                   [-1, 50]          16,050
            Linear-5                   [-1, 10]             510
Total params: 21,840
Trainable params: 21,840
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.06
Params size (MB): 0.08
Estimated Total Size (MB): 0.15
----------------------------------------------------------------


## Create Train Function

In [41]:
def trainModel(model):
  # Set the model in training mode
  model.train()

  # Train the model for some number of epochs
  for epoch in range(n_epochs):

      # For every Image - Label pair in this current batch...
      for batch_idx, (inputs, labels) in enumerate(train_loader):
          # Forward pass
          outputs = model(inputs)               # Pass image data into the model
          loss = loss_function(outputs, labels) # Compare prediction to actual answer
          
          # Backward pass and optimization
          optimizer.zero_grad()                 # Clears previous backpropogation results.
          loss.backward()                       # Performs Backpropogation
          optimizer.step()                      # Applies backpropogation results
          
          # Print training status
          if batch_idx % 100 == 0:
              percent_complete = 100. * batch_idx / len(train_loader)
              print(f"Epoch: {epoch} [{batch_idx * len(inputs)}/{len(train_loader.dataset)} ({percent_complete:.0f}%)]\tLoss: {loss.item():.6f}")


## Create Test Function

In [42]:
def testModel(model):
  model.eval()

  correct = 0
  total = 0
  with torch.no_grad():
      for images, labels in test_loader:
          outputs = model(images)
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()

  print('Accuracy on test set: %d %%' % (100 * correct / total))


## Run Training and Testing

In [43]:
trainModel(model)
testModel(model)

Epoch: 0 [0/60000 (0%)]	Loss: 2.303477
Epoch: 0 [6400/60000 (11%)]	Loss: 0.881523
Epoch: 0 [12800/60000 (21%)]	Loss: 1.029139
Epoch: 0 [19200/60000 (32%)]	Loss: 0.445954
Epoch: 0 [25600/60000 (43%)]	Loss: 0.273671
Epoch: 0 [32000/60000 (53%)]	Loss: 0.258224
Epoch: 0 [38400/60000 (64%)]	Loss: 0.345704
Epoch: 0 [44800/60000 (75%)]	Loss: 0.401346
Epoch: 0 [51200/60000 (85%)]	Loss: 0.372374
Epoch: 0 [57600/60000 (96%)]	Loss: 0.387496
Epoch: 1 [0/60000 (0%)]	Loss: 0.385967
Epoch: 1 [6400/60000 (11%)]	Loss: 0.221749
Epoch: 1 [12800/60000 (21%)]	Loss: 0.480937
Epoch: 1 [19200/60000 (32%)]	Loss: 0.250368
Epoch: 1 [25600/60000 (43%)]	Loss: 0.257325
Epoch: 1 [32000/60000 (53%)]	Loss: 0.443058
Epoch: 1 [38400/60000 (64%)]	Loss: 0.338358
Epoch: 1 [44800/60000 (75%)]	Loss: 0.319236
Epoch: 1 [51200/60000 (85%)]	Loss: 0.136328
Epoch: 1 [57600/60000 (96%)]	Loss: 0.299989
Epoch: 2 [0/60000 (0%)]	Loss: 0.208063
Epoch: 2 [6400/60000 (11%)]	Loss: 0.425756
Epoch: 2 [12800/60000 (21%)]	Loss: 0.232054
Epoch: